In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [27]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Load and process dataset
load breast_cancer.csv, drop columns "id" and "Unnamed: 32", investigate the dataset, and divide into train and test with 80/20 ratio, map values of "diagnosis" from ("B","M") to (0,1)

In [28]:
original_data = pd.read_csv('breast_cancer.csv')
X = original_data.drop(['id', 'Unnamed: 32'], axis=1)
target_col = 'diagnosis'
X.loc[X[target_col] == 'M', 'diagnosis'] = 1
X.loc[X[target_col] == 'B', 'diagnosis'] = 0
X[target_col] = X[target_col].astype(int)

setting number of clusters

In [29]:
K = 2

In [30]:
print('CORRELATION MATRIX:')
for feature in X.columns.difference([target_col]):
    print(f'Correlation between {feature} and target column: ', X[[feature, target_col]].corr().iloc[1,0])

CORRELATION MATRIX:
Correlation between area_mean and target column:  0.7089838365853909
Correlation between area_se and target column:  0.5482359402780249
Correlation between area_worst and target column:  0.7338250349210516
Correlation between compactness_mean and target column:  0.5965336775082529
Correlation between compactness_se and target column:  0.2929992442488583
Correlation between compactness_worst and target column:  0.5909982378417925
Correlation between concave points_mean and target column:  0.7766138400204361
Correlation between concave points_se and target column:  0.40804233271650514
Correlation between concave points_worst and target column:  0.7935660171412696
Correlation between concavity_mean and target column:  0.6963597071719053
Correlation between concavity_se and target column:  0.2537297659808306
Correlation between concavity_worst and target column:  0.6596102103692344
Correlation between fractal_dimension_mean and target column:  -0.012837602698432364
Corr

In [31]:
y = X[target_col]
X.drop(target_col, axis=1, inplace=True)

In [32]:
print('Number of malignant diagnosis: ', y.value_counts().loc[0])
print('Number of benign diagnosis: ', y.value_counts().loc[1])

Number of malignant diagnosis:  357
Number of benign diagnosis:  212


In [33]:
X.describe()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,0.405172,1.216853,2.866059,40.337079,0.007041,0.025478,0.031894,0.011796,0.020542,0.003795,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,0.277313,0.551648,2.021855,45.491006,0.003003,0.017908,0.030186,0.006170,0.008266,0.002646,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,0.111500,0.360200,0.757000,6.802000,0.001713,0.002252,0.000000,0.000000,0.007882,0.000895,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,0.232400,0.833900,1.606000,17.850000,0.005169,0.013080,0.015090,0.007638,0.015160,0.002248,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,0.324200,1.108000,2.287000,24.530000,0.006380,0.020450,0.025890,0.010930,0.018730,0.003187,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,0.478900,1.474000,3.357000,45.190000,0.008146,0.032450,0.042050,0.014710,0.023480,0.004558,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,2.873000,4.885000,21.980000,542.200000,0.031130,0.135400,0.396000,0.052790,0.078950,0.029840,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [34]:
X.mean()

radius_mean                 14.127292
texture_mean                19.289649
perimeter_mean              91.969033
area_mean                  654.889104
smoothness_mean              0.096360
compactness_mean             0.104341
concavity_mean               0.088799
concave points_mean          0.048919
symmetry_mean                0.181162
fractal_dimension_mean       0.062798
radius_se                    0.405172
texture_se                   1.216853
perimeter_se                 2.866059
area_se                     40.337079
smoothness_se                0.007041
compactness_se               0.025478
concavity_se                 0.031894
concave points_se            0.011796
symmetry_se                  0.020542
fractal_dimension_se         0.003795
radius_worst                16.269190
texture_worst               25.677223
perimeter_worst            107.261213
area_worst                 880.583128
smoothness_worst             0.132369
compactness_worst            0.254265
concavity_wo

finding feature which should be rescaled

In [35]:
X_max = X.max()
features_to_rescale = X_max[np.abs(X.max()) > 2].index.tolist()
X[features_to_rescale] = StandardScaler().fit_transform(X[features_to_rescale])


## Implementing KMeans

In [36]:
class KMeans(object):
    def __init__(self, K, metric='L2', max_iter=200, eps=1e-4, center_init='random'):
        self.K = K
        self.max_iter = max_iter
        self.eps = eps
        self.centroids = np.array([])
        self.metric = metric.lower()
        self.center_init = center_init.lower()

        """
        if metric is 'L2' let self.dist be a function that computes euclidian distance between x and y vectors,
        if metric is 'L1' let self.dist be a function that computes manhattan distance between x and y vectors,
        otherwise raise not implemented error
        """
        if self.metric == 'l2':
            self.dist = self.l2_dist
        elif self.metric == 'l1':
            self.dist = self.l1_dist
        else:
            raise NotImplementedError

    def __str__(self):
        return f'KMeans object: metric={self.metric}, center_init={self.center_init}, K={self.K}, max_iter={self.max_iter}, eps={self.eps}'

    def distortion(self, X, r):
        """
        param X: numpy array of shape (M,N)
        param r: numpy array of shape (M), shows to which cluster each row of X belongs
        return: distortion value of the dataset
        """
        sum_ = 0
        for k in range(self.K):
            mask = r[:, k] == 1
            X_k = X[mask]
            sum_ += np.sum(self.dist(X_k, self.centroids[k]))
        print('distortion: ', sum_)
        return sum_

    def init_centroids(self, X):
        """
        :param X: numpy array of shape (M,N)
        """
        """ 
        If centers_init is 'random' initialize self.centroids with random K items from X,
        if it is 'kmeans++' initialize centroids according to the algorithm in 
        http://ilpubs.stanford.edu:8090/778/1/2006-13.pdf page 3,
        otherwise raise not implemented error .
        """
        if self.center_init.lower() == 'random':
            self.centroids = self.random_init(X)
        elif self.center_init.lower() == 'kmeans++':
            self.centroids = self.kmeans_plus_plus_init(X)
        else:
            raise NotImplementedError

    def fit(self, X):
        """
        :param X: numpy array of shape (M,N)
        """
        """ 
        1. Initialize cluster centers using self.init_centroids method
        2. Implement KMeans algorithm and  terminate it when either self.max_iter iterations are performed,
        or the biggest change in cluster centers is smaller than selfk means formula.eps

        The final cluster centers should be saved in self.centroids
        """
        step = 0
        self.init_centroids(X)
        r = self.recalculate_r(X)
        curr_distortion = self.distortion(X, r)

        while step <= self.max_iter:
            r = self.recalculate_r(X)
            self.recalculate_centroids(X, r)

            prev_distortion = curr_distortion
            curr_distortion = self.distortion(X, r)
            if np.abs(prev_distortion - curr_distortion) <= self.eps:
                print(f'Required precision achieved on {step}-th step')
                break

            step += 1
        else:
            print('Maximum iterations run out!')

    def recalculate_centroids(self, X, r):
        for k in range(self.K):
            mask = r[:, k] == 1
            numerator = X[mask].sum(axis=0)
            denominator = r[:, k].sum()
            self.centroids[k] = numerator / denominator

    def recalculate_r(self, X):
        num_rows, num_columns = X.shape
        r = np.zeros(shape=(num_rows, self.K), dtype=int)
        indices = self.find_closest_distances(X, self.centroids)[:, 1].astype('int')
        for i in range(len(indices)):
            r[i, indices[i]] = 1
        return r

    def predict(self, X):
        """
        :param X: numpy array of shape (M,N)
        :return: numpy array of shape (M,)
        """
        """
        using  self.centroids predict to which cluster each datapoint of X belongs, values in returned array
        are integers(id of the cluster). 
        """
        return self.find_closest_distances(X, self.centroids)[:, 1].astype('int')

    def random_init(self, X):
        # for each feature define its boundaries, i.e. minimum and maximum values
        min_boundary = X.min(axis=0)
        max_boundary = X.max(axis=0)

        # return K random vectors of size X.shape[1]
        centroids = np.random.uniform(low=min_boundary, high=max_boundary, size=(self.K, min_boundary.shape[0]))
        return centroids

    def kmeans_plus_plus_init(self, X):
        num_rows, num_columns = X.shape
        # step 1a. Take one center c1, chosen uniformly at random from X
        centroids = np.array(X[np.random.randint(num_rows)])
        centroids = centroids.reshape(-1, len(centroids))

        # step2a.  Take a new center c[i], choosing x ∈ X with probability D(x)**2/sum(D(x)**2)
        for i in range(self.K - 1):
            distances = self.find_closest_distances(X, centroids)[:, 0]
            probabilities = self.get_probabilities(distances)
            max_proba_index = np.argwhere(probabilities == np.amax(probabilities))[0][0]

            # reshape 1d to 2d for appending
            new_centroid = X[max_proba_index].reshape(-1, len(X[max_proba_index]))
            centroids = np.append(centroids, new_centroid, axis=0)
        return centroids

    def get_probabilities(self, distances):
        squared = distances ** 2
        sum_ = np.sum(squared)
        return squared / sum_

    def find_closest_distances(self, X, centroids):
        '''
        :param X:
        :param centroids:
        :return: an array where i-th row is associated with i-th row in X
                 and has two elements: closest distance to centroid and index of that centroid
        '''

        num_rows = X.shape[0]
        closest_distances = np.zeros(shape=(num_rows, 2))

        for i in range(num_rows):
            # array of distances between current point and centroids
            distances = self.dist(centroids, X[i])
            # index of min element in distances assigned to indices array
            min_distance = np.amin(distances)
            closest_distances[i] = min_distance, np.argwhere(distances == min_distance)
        return closest_distances

    def l2_dist(self, X, Y):
        return np.sqrt(np.sum((X - Y) ** 2, axis=1))

    def l1_dist(self, X, Y):
        return np.sum(np.abs(X - Y), axis=1)
    
    

## Cluster the dataset with kmeans, model and predict malignancy of tumors in the test set entries
## 1. Perform clustering using the following hyperparameter pairs
1. metric='L1', center_init='random'
2. metric='L1', center_init='kmeans++'
3. metric='L2', center_init='random'
4. metric='L2', center_init='kmeans++'

## 2. Predict malignancy of tumors in the test set entries using all 4 models trained above, compare their performances.


In [37]:
clf1 = KMeans(K=2, metric='L1', center_init='random')
clf2 = KMeans(K=2, metric='L1', center_init='kmeans++')
clf3 = KMeans(K=2, metric='L2', center_init='random')
clf4 = KMeans(K=2, metric='L2', center_init='kmeans++')

In [38]:
print(clf1)
clf1.fit(X.values)
clusters = clf1.predict(X.values)

labels = np.zeros_like(clusters)
for i in range(2):
    mask = (clusters == i)
    labels[mask] = mode(y[mask])[0]

print(f'accuracy_score: ', accuracy_score(y, labels))

KMeans object: metric=l1, center_init=random, K=2, max_iter=200, eps=0.0001
distortion:  20669.671994324886
distortion:  4275.327582468298
distortion:  3738.9665274646704
distortion:  3740.4167892420433
distortion:  3742.6761338821498
distortion:  3744.805060625072
distortion:  3744.805060625072
Required precision achieved on 5-th step
accuracy_score:  0.8558875219683656


In [39]:
print(clf2)
clf2.fit(X.values)
clusters = clf2.predict(X.values)

labels = np.zeros_like(clusters)
for i in range(2):
    mask = (clusters == i)
    labels[mask] = mode(y[mask])[0]
print(f'accuracy score: ', accuracy_score(y, labels))

KMeans object: metric=l1, center_init=kmeans++, K=2, max_iter=200, eps=0.0001
distortion:  6129.235803024898
distortion:  5245.3026550558825
distortion:  4954.970007473213
distortion:  4540.023297391273
distortion:  4002.6059842313502
distortion:  3831.7524738843526
distortion:  3791.218926165193
distortion:  3769.4416050589784
distortion:  3760.452902863036
distortion:  3752.665021343435
distortion:  3747.3031014839244
distortion:  3744.805060625072
distortion:  3744.805060625072
Required precision achieved on 11-th step
accuracy score:  0.8558875219683656


In [40]:
print(clf3)
clf3.fit(X.values)
clusters = clf3.predict(X.values)

labels = np.zeros_like(clusters)
for i in range(2):
    mask = (clusters == i)
    labels[mask] = mode(y[mask])[0]
print(f'accuracy score: ', accuracy_score(y, labels))

KMeans object: metric=l2, center_init=random, K=2, max_iter=200, eps=0.0001
distortion:  6351.107274480177
distortion:  1446.6337259364627
distortion:  1246.5453451155147
distortion:  1242.4712903325444
distortion:  1240.6426333605189
distortion:  1240.6426333605189
Required precision achieved on 4-th step
accuracy score:  0.8576449912126538


In [41]:
print(clf4)
clf4.fit(X.values)
clusters = clf4.predict(X.values)

labels = np.zeros_like(clusters)
for i in range(2):
    mask = (clusters == i)
    labels[mask] = mode(y[mask])[0]
print(f'accuracy score: ', accuracy_score(y, labels))

KMeans object: metric=l2, center_init=kmeans++, K=2, max_iter=200, eps=0.0001
distortion:  1789.1356696247522
distortion:  1643.3020772902382
distortion:  1614.291967968212
distortion:  1519.7004715046983
distortion:  1376.346206911612
distortion:  1272.6541912846237
distortion:  1251.2569701298153
distortion:  1243.282433176807
distortion:  1240.6426333605189
distortion:  1240.6426333605189
Required precision achieved on 8-th step
accuracy score:  0.8576449912126538


## Fit your implementation of Logistic Regression on the dataset, predict on test set and compare the results with kmeans approach

## Analyze the coefficients of fitted logistic regression model, drop 2 most unimportant features and train again Logistic regression and Kmeans with best metric, center_init hyperparameters, evaluate and compare results

## Analyze the coefficients of fitted initial logistic regression model(using all features), select two most important features and train again Logistic regression and Kmeans with best metric, center_init hyperparameters, evaluate and compare results, make the following plot using the test set:

datapoints with cluster centers and decision boundary, color the datapoints according to Kmeans predictions
color the datapoints on which predictions of logistic regression and Kmeans disagree with separate color


## Compare performance of best Kmeans model with the performance of Kmeans in sklearn library, using the same hyperparameters.